# Машинное обучение, DS-поток
## Задание 1.11


**Правила:**

* Выполненную работу нужно отправить телеграм-боту `@miptstats_ad21_bot`.
* Дедлайны см. в боте. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Прислать нужно ноутбук в формате `ipynb`.
* Решения, размещенные на каких-либо интернет-ресурсах не принимаются. Публикация решения может быть приравнена к предоставлении возможности списать.
* Для выполнения задания используйте этот ноутбук в качестве основы, ничего не удаляя из него.

---

## Генерация текстов с использованием RNN.

**Части задания.** 

* Написание модели, лосса и кода для обучения (7 баллов)

* Реализация жадного алгоритма генерации (3 балла)

* Одно из:

  * Реализация Top k sampling (2 балла)

  * Реализация Beam-search (6 баллов)


В данном задании вы будете генерировать тексты на основании высказываний [Ницше](https://ru.wikipedia.org/wiki/Ницше,_Фридрих) при помощи рекуррентных сетей. 
Модель будет основываться на токенах — словах или более продвинутых способах работы с текстом.



In [1]:
import io
import os
import sys
import random
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F

from nltk.tokenize import TweetTokenizer

from sklearn.model_selection import train_test_split

import time
from IPython.display import clear_output

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

Скачиваем данные. Корпус состоит из высказываний Ницше, разделеных переносом строки.

In [4]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1QefWGfvcn0CXzmd42ySFTkwd1shHTUA3'

"wget" не является внутренней или внешней
командой, исполняемой программой или пакетным файлом.
"id" не является внутренней или внешней
командой, исполняемой программой или пакетным файлом.


In [ ]:
!head nietzsche.txt

PREFACE


SUPPOSING that Truth is a woman--what then? Is there not ground
for suspecting that all philosophers, in so far as they have been
dogmatists, have failed to understand women--that the terrible
seriousness and clumsy importunity with which they have usually paid
their addresses to Truth, have been unskilled and unseemly methods for
winning a woman? Certainly she has never allowed herself to be won; and
at present every kind of dogma stands with sad and discouraged mien--IF,


Посмотрим сколько уникальных слов есть в корпусе.
Также поставим каждому слову (токену) в соответствие число, чтобы далее работать с числами.

In [2]:
with io.open('nietzsche.txt', encoding='utf-8') as f:
    text = f.read().lower()

tokenizer = TweetTokenizer()
tokens = list(set(tokenizer.tokenize(text)))
print('total tokens:', len(tokens))

token_indices = {c: i for i, c in enumerate(tokens)}
indices_token = {i: c for i, c in enumerate(tokens)}

total tokens: 11393


11393 уникальных слов — это относительно много, так как придется на каждом шаге решать задачу классификации на 11393 класса. А брать софтмакс от вектора такой большой размерности затруднительно. Поэтому имеется несколько вариантов: простой (1), очень простой (2) и современный (3) .

1. Отбросить редко встречающиеся слова (токены) и заменить их все на токен \<UNK\>. 

2. Использовать в качестве токенов символы, а не слова. Такую модель мы использовали на семинаре для генерации имён.
Этот подход имеет явный недостаток — модели, обучаемой на отдельных символах, гораздо сложнее выучить соотношения между буквами в каждом отдельном слове. Из-за этого может возникнуть ситуация, при которой значительная часть слов, сгенерированных моделью, не будут являться словами языка, на котором написаны входные тексты.

3. Использовать [Byte Pair Encoding](https://arxiv.org/pdf/1508.07909.pdf). [Одна из опен-сорсных реализаций](https://github.com/VKCOM/YouTokenToMe) от ребят из VK.

Существуют также подходы для аппроксимации softmax, такие как иерархический софтмакс и negative sampling.

Разберём, как работает BPE на примере строки `aaabdaaabac`.
Пара `aa` встречается чаще всего, поэтому она будет заменяется символом `Z`, который не используется в данных, получается строка `ZabdZabac`.
Далее процесс повторяется для пары `ab`, которая заменяется на символ `Y`. Теперь строка имеет вид `ZYdZYac`, причем единственная оставшаяся пара исходных символов встречается только один раз. На этом кодировку можно остановить или же продолжать рекурсивно заменив `ZY` на `X`: `XdXac`. Полученную строку нельзя продолжать сжимать, поскольку не существует пары символов, встречающихся более одного раза. Для декодирования нужно выполнить замены в обратном порядке.

Если же мы будем проделывать эту процедуру не над маленькой строкой, а над большим датасетом, то мы найдем часто встречающиеся n-gram-ы из символов в датасете. Полученные замены будем использовать как новые токены и представлять каждое слово как последовательность таких токенов.
В данном случае получится, что частые слова скорей всего будут представлены лишь одним токеном, равным самому слову. А редкие слова разобъются на последовательность токенов, являющихся частыми.


![](https://docs.google.com/uc?export=download&id=1bD-VA4EWfmMsqFT4ufIhCMDh2fKlP_rC)

Метод BPE является некоторой золотой серединой между character-based и word-based подходами.
[`YouTokenToMe`](https://github.com/VKCOM/YouTokenToMe) — одна из опен-сорсных реализаций от разработчиков из VK. 
Также можно посмотреть [статью](https://arxiv.org/abs/1910.13267) студента ФИВТа с конференции ACL 2020 по bpe-dropout.

Ставим библиотеку, речь о которой шла выше:

In [6]:
! vs_buildtools.exe --norestart --passive --downloadThenInstall --includeRecommended --add Microsoft.VisualStudio.Workload.NativeDesktop --add Microsoft.VisualStudio.Workload.VCTools --add Microsoft.VisualStudio.Workload.MSBuildTools

"vs_buildtools.exe" не является внутренней или внешней
командой, исполняемой программой или пакетным файлом.


In [3]:
! pip install youtokentome

  Using cached youtokentome-1.0.6.tar.gz (86 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for youtokentome
Failed to build youtokentome
  Running setup.py install for youtokentome: started
  Running setup.py install for youtokentome: finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [22 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-39
  creating build\lib.win-amd64-cpython-39\youtokentome
  copying youtokentome\youtokentome.py -> build\lib.win-amd64-cpython-39\youtokentome
  copying youtokentome\yttm_cli.py -> build\lib.win-amd64-cpython-39\youtokentome
  copying youtokentome\__init__.py -> build\lib.win-amd64-cpython-39\youtokentome
  running build_ext
  building '_youtokentome_cython' extension
  creating build\temp.win-amd64-cpython-39
  creating build\temp.win-amd64-cpython-39\Release
  creating build\temp.win-amd64-cpython-39\Release\youtokentome
  creating build\temp.win-amd64-cpython-39\Release\youtokentome\cpp
  "C:\Program Files (x86)\Microsoft Visual Studio\2019\BuildTools\VC\Tools\MSVC\14.29.30133\bin\HostX86\x64\cl.exe" /c /nologo /O2 /W3 /GL /DNDEBUG /MD

Создаем BPE-модель, которую обучим на наших токенах

In [3]:
import youtokentome as yttm

# Зададим желаемый размер словаря после BPE.
# BPE перестанет объединять символы и токены в тот момент, 
# когда текущее количество токенов >= vocab_size
vocab_size = 1000

def get_bpe(tokens, vocab_size=vocab_size):
    """ 
    Возвращает токенизатор BPE, обученный на токенах.
    Параметры.
    1) tokens - токены,
    2) vocab_size - количество уникальных токенов в итоговом словаре.
    """

    with open('tmp.json', 'w', encoding='utf-8') as file_:
        for token in tokens:
            print(token, file=file_)

    yttm.BPE.train('tmp.json', vocab_size=vocab_size, model="bpe.model")
    os.remove('tmp.json')

    return yttm.BPE(model="bpe.model")


In [4]:
bpe = get_bpe(tokens)

In [5]:
bpe.subword_to_id('pos')

268

Пример работы:

In [6]:
example = text.split('\n')[3]

print('Original: ', example, '\n')
print('BPE decoded: ', *bpe.encode(
    example, output_type=yttm.OutputType.SUBWORD, bos=True, eos=True
    ), '\n', sep=' ')
print('BPE encoded: ', bpe.encode(
    example, output_type=yttm.OutputType.ID, bos=True, eos=True
    ))

Original:  supposing that truth is a woman--what then? is there not ground 

BPE decoded:  <BOS> ▁sup pos ing ▁th at ▁truth ▁is ▁a ▁w oman --what ▁th en ? ▁is ▁there ▁not ▁gr ound <EOS> 

BPE encoded:  [2, 597, 268, 69, 153, 79, 871, 772, 86, 101, 595, 712, 153, 64, 55, 772, 612, 763, 228, 267, 3]


Напишите нейронную сеть, использующую RNN, которая должна обучиться на *каждом шаге* предсказывать следующий токен. Для этого вам понадобится к каждому скрытому состоянию RNN применить некоторую классификационную нейронную сеть. На каждом этапе времени такая сеть будет пытаться предсказать следующий токен. На каждом этапе времени должно получиться свое предсказание вероятности для следующего токена. После этого необходимо усреднить лосс по полученным предсказаниям.

**Замечания** <br>
1). Проследите, что вы действительно предсказываете следующий символ, а не текущий. Для этого можно поступить следующим образом.

Пусть на вход приходит последовательность $x_1, x_2, ..., x_{n}$. Требуется на каждом этапе времени $t$ по символам $x_1, ..., x_{t - 1}$ предсказывать символ $x_t$. Для этого на вход нужно подать последовательность \<BOS\>, $x_1, x_2..., x_{n}$ и в качестве таргетов взять последовательность $x_1, x_2, ..., x_{n}, $ \<EOS\>. В данном случае \<BOS\> (begin of sentence), \<EOS\>(end of sentence) — специальные символы начала и конца предложения (любые новые символы, которых нет в словаре). 

2). При обучении вы подаете на вход истинные токены и предсказываете следующий токен, но при тестировании истинных токенов нет. Поэтому при тестировании стоит поступить следующим образом.
* Подать на вход \<BOS\>, применить рекуррентную ячейку и предсказать вероятности быть следующим символом для каждого токена из словаря. 
* Найти токен с максимальной вероятностью. Обозначим его $x_1$. 
* Подать символ $x_1$ на вход рекуррентной ячейке и предсказать вероятности для следующих символов. 
* И так далее. Генерация продолжается до тех пор, пока не был предсказан токен \<EOS\> или мы не достигли нужной `length_to_predict` длины сгенерированного текста, где `length_to_predict` — число, заданное заранее.

Заметим, что при наличии начальной последовательности $x_1, .., x_k$, которую нужно продолжить, сначала на вход нужно подавать истинные символы, а затем, когда истинные закончатся, подавать предсказанные символы. То есть в этом случае мы подаем \<BOS\>, $x_1, .., x_k$ на вход и уже потом начинаем генерацию.

На рисунке ниже можно увидеть пример такой сети. При обучении на вход подаются истинные токены. На этапе времени $t - 1$ предсказываются вероятности $\widehat{y}_{i}$ быть следующим ($i$-ым) токеном для всех токенов из словаря. 
В качестве лосса в данном случае рассматривается функция $\frac{1}{n} \sum\limits_{i=1}^{n} H(y_i, \widehat{y}_i)$, где $H(\mathsf{P}, \mathsf{Q})$ — кросс-энтропия, а $y^i$ — вырожденное распределение, соответствующее истинному индексу $i$-ого токена.

На этапе применения на вход рекуррентной ячейке подается не настоящий токен, а тот токен, который имел наибольшую вероятность быть следующим на предыдущем шаге.

![img](https://i.ibb.co/BnftCLh/Screenshot-2019-05-03-at-21-45-01.png)

![imh](https://i.ibb.co/rwHyYsK/Screenshot-2019-05-03-at-21-44-53.png)

Получим таргеты для обучения. Для этого к каждой строке, т.е. последовательности bpe-токенов добавим токен '\<EOS\>'. Также, сделаем так, чтобы все строки были одной и той же длины, добавив паддинг из символов '\<PAD\>'.

Таким образом, строка $x_1, x_2, ..., x_{8}$ должна преобразиться в $x_1, x_2, ..., x_{8}$, \<EOS\>, \<PAD\>, \<PAD\>, \<PAD\> в случае если мы хотим, чтобы длина всех строк была 12.

После данных преобразований переведите полученные строки в последовательность чисел, с которыми уже далее будет работать модель. Для этого можно использовать метод `bpe.subword_to_id`

Для этого сначала найдем максимальную длину строки.

In [7]:
length = []

for string in text.split('\n'):
    bpe_string = bpe.encode(string, output_type=yttm.OutputType.SUBWORD, bos=False, eos=False)
    if len(bpe_string) > 0:
        length.append(len(bpe_string))
print("Максимальная длина строки:", np.max(length))

Максимальная длина строки: 40


Посчитаем таргеты. Учитывайте, что так как мы добавляем символ \<EOS\>, то полученная длина строки должна быть на 1 больше.

*Замечание* 

После разбиения текста `text` на строки стоит проверить, что все строки имеют ненулевую длину. Строки нулевой длины нужно выкинуть.

In [8]:
MAX_LENGTH = np.max(length) + 1

target_strings = []

for string in text.split('\n'):
    bpe_string = bpe.encode(string, output_type=yttm.OutputType.SUBWORD, bos=False, eos=False)
    if len(bpe_string) > 0:
        bpe_string.append('<EOS>')
        while len(bpe_string) < MAX_LENGTH:
            bpe_string.append('<PAD>')
        target_strings.append(bpe_string)

In [9]:
target_strings[0][:5]

['▁pre', 'f', 'ace', '<EOS>', '<PAD>']

In [10]:
print(np.unique(target_strings))

['!' '"' "'" "'s" '(' ')' ',' '-' '--' '--a' '--and' '--as' '--b' '--but'
 '--d' '--for' '--h' '--he' '--i' '--in' '--is' '--it' '--m' '--n'
 '--namely' '--or' '--s' '--th' '--that' '--the' '--they' '--this' '--to'
 '--w' '--we' '--wh' '--what' '--who' '-b' '-c' '-d' '-f' '-g' '-h' '-l'
 '-like' '-m' '-mor' '-n' '-p' '-r' '-re' '-s' '-t' '-w' '.' '0' '1' '2'
 '3' '4' '5' '6' '7' '8' '9' ':' ';' '<EOS>' '<PAD>' '=' '?' '[' ']' '_'
 'a' 'ab' 'ability' 'able' 'ably' 'ac' 'ace' 'ach' 'ack' 'acr' 'acy' 'ad'
 'ade' 'adic' 'ads' 'af' 'ag' 'age' 'ages' 'aid' 'ail' 'ain' 'ained'
 'aining' 'ains' 'air' 'ak' 'ake' 'aking' 'al' 'ality' 'aliz' 'all' 'ally'
 'als' 'alth' 'am' 'ame' 'amely' 'ament' 'an' 'anc' 'ance' 'ances' 'and'
 'ands' 'ane' 'ang' 'ank' 'ann' 'ans' 'ant' 'ants' 'ap' 'aph' 'app' 'aps'
 'ar' 'ard' 'are' 'arily' 'arm' 'arn' 'ars' 'art' 'ary' 'as' 'ase' 'ash'
 'ason' 'ass' 'ast' 'aste' 'at' 'ate' 'ated' 'ately' 'ates' 'ath' 'ather'
 'ati' 'atic' 'ating' 'ation' 'ations' 'atis' 'atisf' 

In [11]:
# запишем в tokens все возможные токены
tokens = np.unique(target_strings).tolist()
tokens.append('<BOS>')

num_tokens = len(tokens)
print('число токенов =', num_tokens)

число токенов = 996


In [12]:
token_to_id = {token: idx for idx, token in enumerate(tokens)}
print(token_to_id)

{'!': 0, '"': 1, "'": 2, "'s": 3, '(': 4, ')': 5, ',': 6, '-': 7, '--': 8, '--a': 9, '--and': 10, '--as': 11, '--b': 12, '--but': 13, '--d': 14, '--for': 15, '--h': 16, '--he': 17, '--i': 18, '--in': 19, '--is': 20, '--it': 21, '--m': 22, '--n': 23, '--namely': 24, '--or': 25, '--s': 26, '--th': 27, '--that': 28, '--the': 29, '--they': 30, '--this': 31, '--to': 32, '--w': 33, '--we': 34, '--wh': 35, '--what': 36, '--who': 37, '-b': 38, '-c': 39, '-d': 40, '-f': 41, '-g': 42, '-h': 43, '-l': 44, '-like': 45, '-m': 46, '-mor': 47, '-n': 48, '-p': 49, '-r': 50, '-re': 51, '-s': 52, '-t': 53, '-w': 54, '.': 55, '0': 56, '1': 57, '2': 58, '3': 59, '4': 60, '5': 61, '6': 62, '7': 63, '8': 64, '9': 65, ':': 66, ';': 67, '<EOS>': 68, '<PAD>': 69, '=': 70, '?': 71, '[': 72, ']': 73, '_': 74, 'a': 75, 'ab': 76, 'ability': 77, 'able': 78, 'ably': 79, 'ac': 80, 'ace': 81, 'ach': 82, 'ack': 83, 'acr': 84, 'acy': 85, 'ad': 86, 'ade': 87, 'adic': 88, 'ads': 89, 'af': 90, 'ag': 91, 'age': 92, 'ages': 

In [13]:
print(len(target_strings[1]))

41


In [14]:
def to_matrix(strings):
    matrix = np.array(np.empty(shape=(0, len(strings[0])), dtype=np.int_))
    for str in strings:
        line_str = []
        for c in str:
            line_str.append(token_to_id[c])
        matrix = np.append(matrix, [np.array(line_str, dtype=np.int_)], axis=0)
    return matrix

In [15]:
print(to_matrix(target_strings[:5]))

[[888 261  81  68  69  69  69  69  69  69  69  69  69  69  69  69  69  69
   69  69  69  69  69  69  69  69  69  69  69  69  69  69  69  69  69  69
   69  69  69  69  69]
 [944 518 335 956 148 965 826 661 986 451  36 956 220  71 826 959 856 789
  490  68  69  69  69  69  69  69  69  69  69  69  69  69  69  69  69  69
   69  69  69  69  69]
 [779 947 511 561 956 148 674 878 247   6 812 928 774 132 684 957 620 792
  173 692 220  68  69  69  69  69  69  69  69  69  69  69  69  69  69  69
   69  69  69  69  69]
 [729 439 417 160 573   6 792 173 774  95 206 951 430 971 248 119 986 450
  220  28 957 955 528 287  68  69  69  69  69  69  69  69  69  69  69  69
   69  69  69  69  69]
 [920 347 631 119 706 591 541 809 476 382 990 988 291 957 620 792 173 978
  577 108 870  94  68  69  69  69  69  69  69  69  69  69  69  69  69  69
   69  69  69  69  69]]


In [16]:
target_matrix = to_matrix(target_strings)

Получим то, что подается на вход нейросети. Из таргет-строк получите строки тех же длин, которые начинаются с символа \<BOS\> и будут без последнего символа таргет-строки. Полученные строки также должны состоять из чисел, как и ранее посчитанные таргет-строки.

In [17]:
print(target_strings[0][:-1])

['▁pre', 'f', 'ace', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']


In [18]:
feature_strings = []
for str in target_strings:
    feature_str = ['<BOS>'] + str[:-1]
    feature_strings.append(feature_str)
    
print(feature_strings[:5])

[['<BOS>', '▁pre', 'f', 'ace', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>'], ['<BOS>', '▁sup', 'pos', 'ing', '▁th', 'at', '▁truth', '▁is', '▁a', '▁w', 'oman', '--what', '▁th', 'en', '?', '▁is', '▁there', '▁not', '▁gr', 'ound', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>'], ['<BOS>', '▁for', '▁sus', 'pec', 'ting', '▁th', 'at', '▁all', '▁philosoph', 'ers', ',', '▁in', '▁so', '▁f', 'ar', '▁as', '▁the', 'y', '▁h', 'ave', '▁be', 'en', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', 

In [19]:
feature_matrix = to_matrix(feature_strings)
feature_matrix[:5]

array([[995, 888, 261,  81,  68,  69,  69,  69,  69,  69,  69,  69,  69,
         69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,
         69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,
         69,  69],
       [995, 944, 518, 335, 956, 148, 965, 826, 661, 986, 451,  36, 956,
        220,  71, 826, 959, 856, 789, 490,  68,  69,  69,  69,  69,  69,
         69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,
         69,  69],
       [995, 779, 947, 511, 561, 956, 148, 674, 878, 247,   6, 812, 928,
        774, 132, 684, 957, 620, 792, 173, 692, 220,  68,  69,  69,  69,
         69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,
         69,  69],
       [995, 729, 439, 417, 160, 573,   6, 792, 173, 774,  95, 206, 951,
        430, 971, 248, 119, 986, 450, 220,  28, 957, 955, 528, 287,  68,
         69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,
         69,  69],
       [995, 920, 347, 631, 119, 706, 591, 541, 

Разбейте полученные данные на обучение и валидацию.

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train, X_val, y_train, y_val = train_test_split(feature_matrix, target_matrix, test_size=0.2, random_state=42)

Приступим к написанию модели. Ваша модель должна состоять из эмбеддинг слоя, рекуррентного слоя (по желанию, возможно несколько слоев) и линейного слоя, который будет применяться к каждому скрытому состоянию рекуррентного слоя.

В качестве рекуррентного слоя (слоев) лучше использовать `nn.GRU`, `nn.GRUCell`, `nn.LSTM` или `nn.LSTMCell` по вашему желанию.

Напишем 2 модели: простую и модель, которая сначала после эмбеддинга делает одномерные свертки, чтоб модель могла понять, какие признаки лучше, а только потом рекуррентную нейронную сеть. Забегая вперед, скажу, что вторая модель показала потрясающую точность (в районе 60%), но, кажется, это из-за того, что мы фильтром заглядывали вперед. Далее эта модель просто зацикливала слова, то есть, ничего хорошего с нее не получилось. Поэтому будем использовать простую модель.

In [22]:
class SimpleModel(nn.Module):
    ''' Класс модели для генерации имён на основе LSTM, 
    которая принимает на вход предыдущее скрытое состояние '''
    
    def __init__(self, num_tokens=num_tokens, emb_size=64, rnn_num_units=256):
        super(self.__class__, self).__init__()
        self.emb = nn.Embedding(num_tokens, emb_size)
        self.rnn = nn.LSTM(input_size=emb_size, hidden_size=rnn_num_units, batch_first=True)
        self.bn = nn.BatchNorm1d(rnn_num_units)
        self.hid_to_logit = nn.Linear(rnn_num_units, num_tokens)

    def forward(self, x, h0=None, c0=None, device='cuda'):
        # x.size = [batch_size, max_name_len]
        assert isinstance(x.data, torch.LongTensor)

        x = x.to(device)
        # emb.size = [batch_size, max_name_len, emb_size]
        emb = self.emb(x)
        # h_seq.size = [batch_size, max_name_len, rnn_num_units]
        if h0 is not None:
            output, h = self.rnn(emb, (h0, c0))
        else:
            output, h = self.rnn(emb)

        # next_logits.size = [batch_size, max_name_len, num_tokens]
        next_logits = self.hid_to_logit(self.bn(output.permute(0, 2, 1)).permute(0, 2, 1))

        # next_logits.size = [batch_size, max_name_len, num_tokens]
        next_logp = F.log_softmax(next_logits, dim=-1)
        return next_logp, h

In [23]:
class Model(nn.Module):
    ''' Класс модели для генерации имён на основе LSTM, 
    которая принимает на вход предыдущее скрытое состояние '''
    
    def __init__(self, num_tokens=num_tokens, emb_size=256, rnn_num_units=1024, n_filters=256, filter_sizes=[3, 3]):
        super(self.__class__, self).__init__()
        self.emb = nn.Embedding(num_tokens, emb_size)
        self.convs = nn.ModuleList([
          nn.Conv1d(
              in_channels=emb_size, out_channels=n_filters, 
              kernel_size=fs, padding=1,
          ) for fs in filter_sizes
        ])
        self.rnn = nn.LSTM(input_size=len(filter_sizes) * n_filters, hidden_size=rnn_num_units, batch_first=True)
        self.hid_to_logits = nn.Linear(rnn_num_units, num_tokens)
        
    def forward(self, x, h0=None, c0=None, device='cuda'):
        # x.size = [batch_size, max_name_len]
        assert isinstance(x.data, torch.LongTensor)

        x = x.to(device)
        # emb.size = [batch_size, max_name_len, emb_size]
        emb = self.emb(x)
        # emb.size = [batch_size, emb_size, max_name_len]
        emb = emb.permute(0, 2, 1)
        # con.size = [batch_size, n_filters, max_name_len - filter_sizes[n] + 3]
        con = [F.relu(conv(emb)) for conv in self.convs]
        cat = torch.cat(con, dim = 1)
#         con = torch.tensor([F.relu(conv(emb)).cpu().detach().numpy() for conv in self.convs]).cuda()
#         print(cat)
        # con.size = [batch_size, max_name_len - filter_sizes[n] + 3, n_filters]
        cat = cat.permute(0, 2, 1)
        # h_seq.size = [batch_size, max_name_len, rnn_num_units]
        if h0 is not None:
            output, (h, c) = self.rnn(cat, (h0, c0))
        else:
            output, (h, c) = self.rnn(cat)

        # next_logits.size = [batch_size, max_name_len, num_tokens]
        next_logits = self.hid_to_logits(output)

        # next_logits.size = [batch_size, max_name_len, num_tokens]
        next_logp = F.log_softmax(next_logits, dim=-1)
        return next_logp, (h, c)

Напишите функцию вычисления лосса. 

**Замечание.**

Добавленные при помощи паддинга символы не должны оказывать влияния на лосс. То есть по позициям, где истинный символ является специальным символом для паддинга, при вычислении лосса не нужно обращать внимания на предсказания. В противном случае нейронная сеть может научиться всегда предсказывать этот специальный символ из-за того, что он часто встречается. 

Однако, лосс по символу '\<EOS\>' должен быть посчитан и добавлен к финальному лоссу.

Например, можно посчитать лосс следующим образом:

`criterion = nn.CrossEntropyLoss(ignore_index=pad_ix, size_average=True)`

`criterion(logits, y_batch)`

вместо

`F.cross_entropy(logits, y_batch)`

Здесь `pad_ix` — число, которое мы поставили в соответствие специальному символу для паддинга ранее.

In [24]:
def compute_loss(model, X_batch, y_batch):
    pad_idx = token_to_id['<PAD>']
    logits, _ = model(X_batch)
    criterion = nn.CrossEntropyLoss(ignore_index=pad_idx, size_average=True)
    loss = criterion(logits.view(-1, num_tokens), y_batch.view(-1).to(device))
    return loss

Напишем функцию генерации батчей.

In [25]:
def iterate_minibatches(X, y, batch_size, shuffle=True):
    """ Генератор случайных батчей """

    if shuffle:
        indices = np.random.permutation(np.arange(len(X)))
    else:
        indices = np.arange(len(X))
        
    for start in range(0, len(indices), batch_size):
        ix = indices[start: start + batch_size]
        yield X[ix], y[ix]

Инициализируем модель.

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model().to(device)

In [27]:
model = SimpleModel().to(device)

Обучение. Это может занять некоторое время. Постройте также график кривой обучения.

In [31]:
from sklearn.metrics import accuracy_score

In [32]:
def binary_accuracy(preds, y):
    """
    Возвращает точность модели.

    Параметры.
    1) preds — предсказания модели,
    2) y — истинные метки классов.

    """

    binary_preds = np.argmax(preds, axis=2)
    
    # округляет предсказания до ближайшего integer
    acc = accuracy_score(y.reshape(-1), binary_preds.reshape(-1))
    return acc

In [33]:
%%time
opt = torch.optim.Adam(model.parameters(), lr=3e-4)
scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=10, gamma=0.3)

train_loss = []
val_accuracy = []
num_epochs = 30
batch_size = 64

for epoch in range(num_epochs):
    start_time = time.time()
    
    # Обучение
    model.train(True)
    train_loss = []
    for X_batch, y_batch in iterate_minibatches(X_train, y_train, batch_size, True):
        X_batch = torch.LongTensor(X_batch)
        y_batch = torch.LongTensor(y_batch)
        
        # Считаем функцию потерь
        loss = compute_loss(model, X_batch, y_batch)
        train_loss.append(loss.item())
        
        # делаем backprop
        loss.backward()
        opt.step()
    
    # Валидация
    model.train(False)
    val_acc = []
    with torch.no_grad(): # отключим подсчёт градиентов на валидации
        for X_batch, y_batch in iterate_minibatches(X_val, y_val, batch_size, False):
            X_batch = torch.tensor(X_batch, dtype=torch.int64)
            logits, _ = model(X_batch)
            val_loss = compute_loss(model, X_batch, torch.LongTensor(y_batch))
            scheduler.step(val_loss)
            acc = binary_accuracy(logits.to('cpu').numpy(), y_batch)
            val_acc.append(acc)
            
            
    clear_output(True)
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss (in-iteration): \t{:.6f}".format(
        np.mean(train_loss[-len(X_train) // batch_size :])))
    
    print("  validation accuracy: \t\t\t{:.2f} %".format(
        np.mean(val_acc[-len(X_val) // batch_size :]) * 100))

Epoch 30 of 30 took 1.641s
  training loss (in-iteration): 	3.332850
  validation accuracy: 			16.33 %
CPU times: total: 50.2 s
Wall time: 50.6 s


Результаты не очень радуют. С другой стороны, среди почти 1000 токенов модель угадывает правильно 1 раз из 6, что в принципе сильно лучше, чем если бы мы случайно выбирали некоторый токен. Посмотрим, что будет в другой части задания.

Напишите функцию, которая по началу высказывания будет генерировать следующие `length_to_predict` символов:
* Входной текст приведите к формату, который принимает на вход модель. 
* Предскажите первый символ. 
* Предсказаный символ следует добавить в конец текста и полученный текст опять привести к нужному формату. Альтернативно, вы можете прогнать лишь одну ячейку рекуррентной сети, на вход которой подать только что предсказанный символ и в качестве предыдущего скрытого состояния которой взять скрытое состояние полученное на предыдущем шаге.
* Затем можно предсказать следующий символ и так далее. 


*Замечания.*

1. Модель принимает объекты батчами, поэтому если вы будете подавать ей один элемент она может ругаться. Один элемент нужно подавать как батч размера 1.

2. Для получения предсказания в виде индекса следующего символа нужно найти аргмаксимум по логитам. В этом вам поможет `logits.max(dim)`, который принимает в аргументах размерность `dim` по которой искать максимумы и возвращает сразу 2 тензора — максимумы и индексы максимумов.




Сначала реализуйте жадную генерации текста — на каждом шаге берите токен с наибольшей вероятностью.

In [488]:
def simple_predict(model, text, length_to_predict):
    bpe_string = bpe.encode(text, output_type=yttm.OutputType.SUBWORD, bos=False, eos=False)
    generated_bpe_string = bpe_string.copy()
    generated_matrix = torch.LongTensor(to_matrix([generated_bpe_string]))
    token_id = 0
    logits = []
    while len(generated_matrix[0]) < length_to_predict and generated_matrix[0][-1] != token_to_id['<EOS>']:
        if token_id == 0:
            logp_next, (h, c) = model(generated_matrix)
        else:
            logp_next, (h, c) = model(generated_matrix[-1:], h, c)
        p_next = F.softmax(logp_next[:, -1, :], dim=-1).data.cpu().numpy()[0, :]
        
        next_idx = np.argmax(p_next)
        logits.append(np.log(np.max(p_next)))
        next_idx = torch.LongTensor([[next_idx]])
        generated_matrix = torch.cat([generated_matrix, next_idx], dim=1)
        
    generated_bpe_string = ''.join([tokens[idx] for idx in generated_matrix.data.numpy()[0]])
    return [bpe.subword_to_id(c) for c in generated_bpe_string], logits

Посмотрим, что предсказывает модель. Не пугайтесь, если будет не много смысла в самих высказываниях.

In [205]:
torch.save({'model': model.state_dict(), 'epoch': epoch}, 'rnn_conv.pth')

In [490]:
model.to('cuda')

SimpleModel(
  (emb): Embedding(996, 256)
  (rnn): LSTM(256, 1024, batch_first=True)
  (hid_to_logits): Linear(in_features=1024, out_features=996, bias=True)
)

In [585]:
text, log_probs = simple_predict(model, "be no mistake about it:", 50)
print(text)
bpe.decode([list(np.array(text))])

[4, 24, 5, 4, 8, 12, 4, 18, 6, 7, 9, 10, 27, 5, 4, 10, 24, 12, 16, 9, 4, 6, 9, 44, 4, 58, 14, 5, 25, 5, 13, 12, 17, 16, 18, 6, 8, 19, 56, 1, 1, 1, 1, 1]


['be no mistake about it: "developuming,<UNK><UNK><UNK><UNK><UNK>']

In [583]:
text, log_probs = simple_predict(model, "artificial intelegence is interesting", 50)

bpe.decode([list(np.array(text))])

['artificial intelegence is interesting to the point of view of the<UNK><UNK><UNK><UNK><UNK>']

In [584]:
text, log_probs = simple_predict(model, 'people love ml and', 45)

bpe.decode([list(np.array(text))])

['people love ml and has mere<UNK><UNK><UNK><UNK><UNK>']

Модель часто вставляет неизвестный токен. Также стоит отметить, что даже когда модель его не вставляет, получается белиберда. Будем надеяться, что после Beam-search'а будет лучше.

Жадная генерация не самая хорошая. Возможно ваши предсказания зациклились. Это связано с тем, что 
1. Модель в основном смотрит лишь на последние несколько токенов при предсказании следующего.

2. При предсказании модель генерирует жадно и максимизирует вероятность только одного следующего токена, не думая про то, что может быть дальше.

3. Модель была обучена максимизировать правдоподобие и генерировать токены смотря лишь на предыдущие токены. Из этого модель сильно теряет в разнообразии и может получится так, что модель найдет некоторые часто повторяющиеся структуры в датасете и будет присваивать им большую вероятность. 

Для решения этой проблемы существует несколько методов. Одни из них:

* Top k sampling 

  Для предсказания следующего токена посмотрим на  распределение вероятностей на следующий токен. Выбираем $k$ токенов с максимальной вероятностью и из них сэмплируем один токен с вероятностью пропорциональной предсказанной для этого токена вероятности.

* Beam-search


\\

**Реализуйте один из подходов.** 

* Top k sampling (1 балл)

* Beam-search (3 балла)

Учитывайте, что реализовать beam-search намного сложнее, чем top k sampling, поэтому за его реализацию дается больше баллов.

In [218]:
v = []
v.append([1, 2])
v.append([3, 4])
np.array(v).reshape(-1).tolist()

[1, 2, 3, 4]

In [379]:
b = np.array([1, 2, 3])
idx = [b != 2]
b[idx][:3]

array([1, 3])

In [536]:
def predict(model, text, length_to_predict, k=100, m=10000):
    bpe_string = bpe.encode(text, output_type=yttm.OutputType.SUBWORD, bos=False, eos=False)
    generated_bpe_string = bpe_string.copy()
    generated_matrix = torch.LongTensor(to_matrix([generated_bpe_string]))
    token_id = -1
    beam_search_variants = generated_matrix
    beam_search_probs = []
    best_comb = generated_matrix[0]
    while len(best_comb) < length_to_predict and best_comb[-1] != token_to_id['<EOS>']:
        token_id += 1
        if token_id == 0:
            logp_next, (h, c) = model(generated_matrix)
            p_next = F.softmax(logp_next[:, -1, :], dim=-1).data.cpu().numpy()[0, :]
            top_k_variants = np.flip(np.argsort(p_next))[:k]
            top_k_probs = np.flip(np.sort(p_next))[:k]
            beam_search_variants = beam_search_variants.numpy().reshape(-1)
            beam_search_variants = [np.append(beam_search_variants, var) for var in top_k_variants]
            beam_search_probs = [prob for prob in top_k_probs]
            continue
        old_beam_search_variants = beam_search_variants
        old_beam_search_probs = beam_search_probs
        beam_search_variants = []
        beam_search_probs = []
#         print(old_beam_search_variants)
        for beam_var, beam_prob in zip(old_beam_search_variants, old_beam_search_probs):
            logp_next, (h, c) = model(torch.LongTensor(beam_var[-1:]).reshape(1, -1), h, c)
            p_next = F.softmax(logp_next[:, -1, :], dim=-1).data.cpu().numpy()[0, :]
            top_k_variants = np.flip(np.argsort(p_next))[:k]
            tok_k_probs = np.flip(np.sort(p_next))[:k]
            beam_search_variants.append([np.append(beam_var, var) for var in top_k_variants])
            beam_search_probs.append([beam_prob * prob for prob in top_k_probs])
        beam_search_variants = beam_search_variants[0]
#         print(beam_search_variants)
#         print(beam_search_probs)
        beam_search_probs = np.array(beam_search_probs).reshape(-1).tolist()
#         print(beam_search_probs)
#         print(token_id)
#         print(beam_search_variants)

        if len(beam_search_variants) > m:
            top_m_idxs = np.flip(np.argsort(beam_search_probs))[:m]
            beam_search_variants = beam_search_variants[top_m_idxs]
            beam_search_probs = np.flip(np.sort(beam_search_probs))[:m]
            
        best_comb = beam_search_variants[0]
        
    
    generated_bpe_string = ''.join([tokens[idx] for idx in best_comb])
    return [bpe.subword_to_id(c) for c in generated_bpe_string]

In [537]:
bpe.id_to_subword(4)

'▁'

In [586]:
text = predict(model, "be no mistake about it:", 50)

bpe.decode([list(np.array(text))])

['be no mistake about it: " profound in the thatsimshism age-mid cur forward, the law: they<UNK><UNK><UNK><UNK><UNK>']

In [587]:
text = predict(model, "artificial intelegence is interesting", 50)
bpe.decode([list(np.array(text))])

['artificial intelegence is interesting to in when obediocrever and year the wildisonscil-and thems,<UNK><UNK><UNK><UNK><UNK>']

In [588]:
text = predict(model, 'people love ml and', 50)

bpe.decode([list(np.array(text))])

['people love ml and hasats of the evoach many to accord, as god,<UNK><UNK><UNK><UNK><UNK>']

После beam search'а модель стала иногда предсказывать что-то осмысленное. Хотя проблема с токенами `<UNK>` осталась. В любом случае, beam search работает лучше, чем наивная модель предсказывания.

### **Дополнительная информация**

[Статья](https://www.aclweb.org/anthology/P19-1365.pdf) с ACL 2019 про сравнение различных методов декодирования.